# Exercise 3: Kolizje MD5 i Łamanie Haseł

## Cel ćwiczenia
1. Znalezienie kolizji MD5 (pierwsze 6 znaków)
2. Rozpoznanie schematów hashowania (MD5-crypt, SHA256-crypt, Argon2)
3. Przeprowadzenie ataku słownikowego
4. Obsługa hash'y z pieprzem (salt suffix)

## Część 1: Kolizje MD5

### Czym jest MD5?
**MD5 (Message Digest Algorithm 5)** to funkcja skrótu kryptograficznego produkująca 128-bitową (16-bajtową) wartość skrótu, zazwyczaj reprezentowaną jako 32-znakowy ciąg szesnastkowy.

Właściwości:
- **Deterministyczna**: Ten sam input zawsze daje ten sam output
- **Stały rozmiar**: Zawsze 128 bitów niezależnie od rozmiaru wejścia
- **Jednokierunkowa**: Obliczeniowo niemożliwe do odwrócenia
- **Efekt lawiny**: Mała zmiana inputu drastycznie zmienia output

### Czym jest kolizja?
Kolizja występuje gdy dwie różne wartości wejściowe dają ten sam hash:

```
MD5("hasło1") = abc123...
MD5("hasło2") = abc123...  ← Kolizja!
```

**Dlaczego to ważne**: Łamie założenie unikalności hashy, kompromituje weryfikację integralności.

### Paradoks urodzin
Prawdopodobieństwo znalezienia kolizji rośnie znacznie szybciej niż można by intuicyjnie przypuszczać:

$$P(\text{kolizja}) \approx 1 - e^{-\frac{n^2}{2m}}$$

gdzie:
- $n$ = liczba prób (obliczonych hashy)
- $m$ = liczba możliwych wartości hashy

### Atak częściowej kolizji
W tym ćwiczeniu celujemy w **pierwsze 6 znaków hex** (24 bity):
- Przestrzeń poszukiwań: $m = 16^6 = 16,777,216$ możliwych wartości
- Oczekiwana liczba prób: $\sqrt{\pi m / 2} \approx 4,096$ dla 50% prawdopodobieństwa
- Przy szczęściu: Można znaleźć w ~2,000-5,000 próbach

### Strategia ataku
```python
1. Generuj losowe hasła: "pass0", "pass1", "pass2", ...
2. Oblicz MD5 dla każdego: hash_dict[prefix] = hasło
3. Sprawdź czy prefix już widziany → KOLIZJA!
4. Kontynuuj do znalezienia kolizji
```

**Złożoność**: O(√m) z pamięcią O(n)

## Część 2: Schematy hashowania haseł

### Po co hashować hasła?
Nigdy nie przechowuj haseł w postaci jawnej! Hashowanie zapewnia:
- **Transformację jednokierunkową**: Nie można odzyskać oryginalnego hasła
- **Weryfikację**: Porównaj hash(input) z zapisanym hashem
- **Ochronę przy wycieku**: Nawet gdy baza wycieknie, hasła nie są od razu ujawnione

### Salt: Obrona przed tablicami tęczowymi
**Salt** = Losowa wartość dodana do hasła przed hashowaniem:
```
hash = H(hasło || salt)
```

**Bez salta**: Wstępnie obliczone tablice (rainbow tables) pozwalają na natychmiastowe wyszukiwanie
**Z saltem**: Każde hasło wymaga unikalnego obliczenia, uniemożliwia wstępne obliczenia

### MD5-crypt ($1$)
Format: `$1$salt$hash`

**Struktura**:
- `$1$` → Identyfikator algorytmu (MD5-crypt)
- `salt` → 8-znakowy losowy salt
- `hash` → 22-znakowy hash zakodowany w base64

**Właściwości**:
- Używa 1,000 iteracji MD5
- Wprowadzony w 1994 dla systemów Unix
- **Status**: Przestarzały, zbyt szybki do obliczenia
- **Podatność**: Nowoczesne GPU mogą testować miliony haseł/sekundę

**Przykład**: `$1$abcd1234$e5fskK1p2kLxY.Z9wMFvg/`

### SHA256-crypt ($5$)
Format: `$5$rounds=N$salt$hash`

**Struktura**:
- `$5$` → Identyfikator algorytmu (SHA256-crypt)
- `rounds=N` → Liczba iteracji (opcjonalne, domyślnie 5,000)
- `salt` → Do 16-znakowy salt
- `hash` → 43-znakowy hash zakodowany w base64

**Właściwości**:
- Używa funkcji skrótu SHA-256
- Konfigurowalna liczba iteracji (1,000 do 999,999,999)
- Silniejszy niż MD5-crypt, ale wciąż łamiący się
- **Status**: Akceptowalny dla systemów o niskiej wartości, niezalecany dla nowych wdrożeń

**Przykład**: `$5$rounds=5000$saltsalt$kDV8h2KL7eDL.z2W0nNFKz1oS.xM7dQn`

### Argon2 ($argon2id$)
Format: `$argon2id$v=19$m=65536,t=3,p=4$salt$hash`

**Struktura**:
- `$argon2id$` → Wariant (argon2i, argon2d, lub argon2id)
- `v=19` → Numer wersji
- `m=65536` → Koszt pamięci (KiB)
- `t=3` → Koszt czasu (iteracje)
- `p=4` → Stopień równoległości (wątki)
- `salt` → Salt zakodowany w base64
- `hash` → Hash zakodowany w base64

**Dlaczego Argon2 jest lepszy**:
1. **Memory-hard**: Wymaga dużej ilości RAM, odporne na ataki GPU/ASIC
2. **Konfigurowalny**: Dostosuj czas/pamięć/równoległość do potrzeb bezpieczeństwa
3. **Odporny na ataki kanałami bocznymi**: wariant argon2id chroni przed atakami czasowymi
4. **Zwycięzca**: Konkurs Password Hashing Competition 2015

**Parametry bezpieczeństwa**:
- Minimum: m=15360 (15 MB), t=2, p=1
- Zalecane: m=65536 (64 MB), t=3, p=4
- Wysokie bezpieczeństwo: m=262144 (256 MB), t=5, p=4

**Przykład**: `$argon2id$v=19$m=65536,t=3,p=4$c29tZXNhbHQ$RdescudvJCsgt3ub+b+dWRWJTmaaJObG`

### Tabela porównawcza

| Schemat | Rok | Iteracje | Atak GPU | Status |
|---------|-----|----------|----------|---------|
| MD5-crypt | 1994 | 1,000 | Łatwy | Przestarzały |
| SHA256-crypt | 2008 | 5,000+ | Średni | Akceptowalny |
| Argon2 | 2015 | Konfigurowalny | Odporny | Zalecany |

## Część 3: Atak słownikowy

### Czym jest atak słownikowy?
Systematyczna próba złamania hashy haseł poprzez testowanie popularnych haseł z listy (słownika).

**Przebieg ataku**:
```
1. Załaduj słownik: ["password", "123456", "admin", ...]
2. Dla każdego kandydata na hasło:
   a. Wyodrębnij salt z hash'a docelowego
   b. Oblicz hash(kandydat, salt)
   c. Porównaj z hash'em docelowym
   d. Jeśli pasuje → HASŁO ZNALEZIONE!
```

### Dlaczego ataki słownikowe działają
**Wzorce ludzkich haseł**:
- Używanie popularnych słów, imion, dat
- Przewidywalne zastąpienia (a→@, e→3)
- Krótkie długości (8-12 znaków)
- Ponowne użycie między stronami

**Statystyki**:
- ~80% haseł w wyciekłych bazach jest w popularnych słownikach
- Top 10,000 haseł pokrywa ~30% wszystkich użytkowników
- "password", "123456", "qwerty" pozostają najpopularniejsze

### Złożoność ataku
Dla słownika rozmiaru $D$:
- **Bez pieprzu**: $O(D)$ prób
- **Z 1-znakowym pieprzem [a-z]**: $O(26 \times D)$ prób
- **Z 2-znakowym pieprzem**: $O(26^2 \times D) = O(676 \times D)$ prób

### Pieprz: Dodatkowy tajny składnik
**Pieprz (pepper)** = Tajna wartość doklejana/poprzedzająca hasło przed hashowaniem:
```
hash = H(hasło || pieprz, salt)
```

**Kluczowe różnice od salta**:
| Cecha | Salt | Pieprz |
|-------|------|--------|
| Przechowywanie | Z hashem (publiczny) | Osobna bezpieczna lokalizacja (tajny) |
| Unikalny dla hasła | Tak | Nie (wspólny) |
| Cel | Zapobieganie rainbow tables | Spowolnienie ataków słownikowych |

**W tym ćwiczeniu**:
- Pieprz = pojedynczy znak z [a-z]
- Doklejony do hasła: `"alibaba" + "x"` → `"alibabax"`
- Atakujący musi przetestować wszystkie 26 możliwości dla każdego słowa ze słownika

### Strategia ataku z pieprzem
```python
for hasło in słownik:
    for pieprz in 'abcdefghijklmnopqrstuvwxyz':
        kandydat = hasło + pieprz
        if weryfikuj(kandydat, hash):
            return kandydat
```

**Złożoność**: 26× wolniej, ale wciąż wykonalne dla małej przestrzeni pieprzu

### Techniki optymalizacji
1. **Przetwarzanie równoległe**: Rozdziel słownik między wiele wątków/rdzeni
2. **Akceleracja GPU**: Użyj CUDA/OpenCL dla masywnej równoległości
3. **Inteligentne sortowanie**: Testuj najpopularniejsze hasła jako pierwsze
4. **Mutacje oparte na regułach**: Zastosuj popularne transformacje (wielka litera, dodaj cyfry)

### Szacowany czas (MD5-crypt, pojedynczy rdzeń CPU)
- 10,000 słów w słowniku: ~1-2 sekundy
- 100,000 słów: ~10-20 sekund
- Z mnożnikiem pieprzu 26×: ~5-10 minut
- Argon2 (wysokie parametry): Godziny do dni

## Szczegóły implementacji

### Parsowanie formatów hashy
Każdy schemat ma unikalny identyfikator na początku:

```python
def rozpoznaj_typ_hash(hash_string):
    if hash_string.startswith('$1$'):
        return 'MD5-crypt'
    elif hash_string.startswith('$5$'):
        return 'SHA256-crypt'
    elif hash_string.startswith('$argon2'):
        return 'Argon2'
    else:
        return 'Nieznany'
```

### Wyodrębnianie salta z hasha
**Przykład MD5-crypt**: `$1$abcd1234$e5fskK1p2kLxY.Z9wMFvg/`
```python
części = hash_string.split('$')
# części = ['', '1', 'abcd1234', 'e5fskK1p2kLxY.Z9wMFvg/']
algorytm = części[1]  # '1'
salt = części[2]      # 'abcd1234'
wartość_hash = części[3] # 'e5fskK1p2kLxY.Z9wMFvg/'
```

**SHA256-crypt**: Może zawierać parametr rounds
```python
# Format: $5$rounds=5000$saltsalt$hash...
if 'rounds=' in części[2]:
    rounds_str = części[2]  # 'rounds=5000'
    salt = części[3]        # 'saltsalt'
else:
    salt = części[2]        # Domyślne rundy
```

**Argon2**: Bardziej złożone wyodrębnianie parametrów
```python
# $argon2id$v=19$m=65536,t=3,p=4$salt$hash
wariant = części[1]       # 'argon2id'
wersja = części[2]        # 'v=19'
parametry = części[3]     # 'm=65536,t=3,p=4'
salt = części[4]          # zakodowany base64
wartość_hash = części[5]  # zakodowany base64
```

### Ręczne obliczanie hashy (Passlib)
Zamiast używać `verify()`, ręcznie oblicz hash dla nauki:

```python
from passlib.hash import md5_crypt, sha256_crypt

# Wyodrębnij salt z hash'a docelowego
salt = hash_docelowy.split('$')[2]

# Oblicz hash z tym samym saltem
test_hash = md5_crypt.using(salt=salt).hash(hasło)

# Porównaj pełne hashe
if test_hash == hash_docelowy:
    print(f"Hasło znalezione: {hasło}")
```

### Po co ręczne obliczanie?
1. **Edukacyjne**: Zrozumienie jak weryfikacja działa wewnętrznie
2. **Kontrola**: Zobacz dokładne wartości hashy będące porównywane
3. **Debugowanie**: Łatwiejsze rozwiązywanie problemów
4. **Przejrzystość**: Brak "magii" działającej za kulisami

### Ładowanie słownika
```python
def załaduj_słownik(ścieżka):
    with open(ścieżka, 'r', encoding='utf-8') as f:
        # Wczytaj wszystkie linie, usuń białe znaki, odfiltruj puste
        hasła = [linia.strip() for linia in f if linia.strip()]
    return hasła
```

### Śledzenie postępu (opcjonalne)
Dla dużych słowników, pokaż postęp:
```python
from tqdm import tqdm  # Biblioteka pasków postępu

for hasło in tqdm(słownik, desc="Testowanie haseł"):
    if testuj_hasło(hasło, hash_docelowy):
        return hasło
```

### Obsługa błędów
```python
try:
    wynik = złam_hash(hash_string, słownik)
except ValueError as e:
    print(f"Nieprawidłowy format hash'a: {e}")
except Exception as e:
    print(f"Nieoczekiwany błąd: {e}")
```

## Analiza bezpieczeństwa

### Dlaczego te hash'e są słabe?

#### 1. Słabe wybieranie haseł
**Problem**: Hasła wybrane z popularnego słownika
```
Przykłady: "password", "123456", "admin", "welcome"
```

**Dlaczego to źle**:
- Występują w każdym słowniku atakującego
- Łatwo odgadnąć poprzez brute force
- Brak losowości lub złożoności

**Rozwiązanie**: Używaj **generatorów haseł** z wysoką entropią:
```
Dobre: "X7$mK9#pL2@qR5^wN3&vB8*"
Złe:   "password123"
```

#### 2. Przestarzałe algorytmy (MD5-crypt)
**Problem**: Tylko 1,000 iteracji MD5 (standard z 1994)

**Wydajność współczesnego sprzętu** (pojedyncza GPU):
- MD5: ~200 miliardów hashy/sekundę
- MD5-crypt: ~20 milionów haseł/sekundę
- Czas złamania 8-znakowego hasła [a-z]: ~20 minut

**Rozwiązanie**: Użyj Argon2 z wysokimi parametrami (300× wolniejszy)

#### 3. Niewystarczająca liczba iteracji
**Problem**: SHA256-crypt z domyślnymi 5,000 rundami

**Prędkość ataku**:
- 5,000 rund: ~300,000 haseł/sekundę (GPU)
- 100,000 rund: ~15,000 haseł/sekundę
- Argon2: ~100 haseł/sekundę

**Rozwiązanie**: Dostosuj liczbę iteracji tak aby wymagać ~0.5-1 sekundy na hash

#### 4. Krótka przestrzeń pieprzu
**Problem**: 26 możliwych wartości [a-z]

**Wpływ na atak**:
- Tylko mnożnik 26× na czas ataku
- Dla szybkich hashy (MD5-crypt): sekundy → minuty
- Wciąż łatwo poddaje się brute force

**Rozwiązanie**: Użyj **64+ znakowego losowego pieprzu** z pełnego zakresu ASCII

### Strategie obrony

#### Poziom 1: Edukacja użytkownika
- **Minimum 12 znaków** (zalecenie NIST)
- Mieszaj wielkie, małe litery, cyfry, symbole
- Unikaj słów ze słownika, informacji osobistych
- Używaj menedżerów haseł (1Password, Bitwarden)

#### Poziom 2: Egzekwowanie polityki haseł
```python
Wymagania:
- Minimalna długość: 12 znaków
- Różnorodność znaków: 3+ typy (wielkie, małe, cyfry, symbole)
- Czarna lista: Popularne hasła, nazwa firmy
- Brak ponownego użycia: Sprawdź z poprzednimi 10 hasłami
```

#### Poziom 3: Silny algorytm hashowania
**Używaj Argon2id** z zalecanymi parametrami:
```python
from argon2 import PasswordHasher

ph = PasswordHasher(
    time_cost=3,        # 3 iteracje
    memory_cost=65536,  # 64 MB RAM
    parallelism=4       # 4 wątki
)

hash = ph.hash("hasło_użytkownika")
ph.verify(hash, "hasło_użytkownika")  # Rzuca wyjątek jeśli złe
```

#### Poziom 4: Dodatkowe mechanizmy obrony
1. **Ograniczanie prób**: Max 5 prób logowania na minutę
2. **Blokada konta**: Tymczasowo wyłącz po 10 nieudanych próbach
3. **2FA/MFA**: Wymagaj drugiego czynnika (TOTP, SMS, klucz sprzętowy)
4. **Monitoring**: Alerty przy podejrzanych wzorcach logowania
5. **Pieprz**: Przechowuj tajny pieprz w HSM lub osobnej konfiguracji

### Kompromis czas-pamięć
**Cel bezpieczeństwa**: Spraw by łamanie trwało dłużej niż życie hasła

| Czas hash'a | Prędkość ataku | Czas łamania 8 znaków [a-z] |
|-------------|----------------|-----------------------------|
| 0.001s (MD5-crypt) | 1M/s | 3 godziny |
| 0.01s (SHA256 10K) | 100K/s | 1 dzień |
| 0.1s (Argon2 low) | 10K/s | 3 tygodnie |
| 1.0s (Argon2 high) | 1K/s | 7 miesięcy |

**Zalecenie**: Cel 0.5-1 sekunda na hash na sprzęcie produkcyjnym

### Przykłady rzeczywistych wycieków
- **LinkedIn (2012)**: 6.5M hashy SHA-1 (bez salta!) złamane w dni
- **Adobe (2013)**: 150M haseł z słabym szyfrowaniem
- **Yahoo (2013)**: 3 miliardy kont, bcrypt (lepszy rezultat)
- **RockYou (2009)**: 32M haseł w plain text (katastrofa!)

### Kluczowe wnioski
1. ✅ Zawsze używaj **Argon2** dla nowych systemów
2. ✅ Nigdy nie przechowuj plain text lub odwracalnego szyfrowania
3. ✅ Unikalny salt dla każdego hasła, generowany kryptograficznie
4. ✅ Długi pieprz (64+ znaków), przechowywany osobno
5. ✅ Egzekwuj silne polityki haseł
6. ✅ Implementuj ograniczanie prób i 2FA
7. ❌ Nigdy nie używaj MD5, SHA-1, lub zwykłego SHA-256 do haseł
8. ❌ Nie twórz własnej kryptografii

## Praktyczny przegląd

### Przepływ pracy ćwiczenia

#### Krok 1: Znajdź kolizję MD5
```python
import hashlib

def znajdź_kolizję_md5(długość_prefiksu=6):
    widziane = {}
    licznik = 0
    
    while True:
        hasło = f"pass{licznik}"
        hash_pełny = hashlib.md5(hasło.encode()).hexdigest()
        prefix = hash_pełny[:długość_prefiksu]
        
        if prefix in widziane:
            # Kolizja znaleziona!
            return widziane[prefix], hasło, prefix
        
        widziane[prefix] = hasło
        licznik += 1
```

**Oczekiwany wynik**:
```
Kolizja znaleziona!
Hasło 1: pass862
Hasło 2: pass4708
Wspólny prefix: b75278
```

#### Krok 2: Złam hash MD5-crypt
Dano: `$1$saltyyyy$kR8qLKZw7LJ3ZNXhzN5Ql.`

```python
from passlib.hash import md5_crypt

def złam_md5_crypt(hash_docelowy, słownik):
    salt = hash_docelowy.split('$')[2]  # 'saltyyyy'
    
    for hasło in słownik:
        test_hash = md5_crypt.using(salt=salt).hash(hasło)
        
        if test_hash == hash_docelowy:
            return hasło  # Znalezione: "alibaba"
    
    return None
```

#### Krok 3: Złam hash SHA256-crypt
Dano: `$5$rounds=5000$abcdefgh$xyz...`

```python
from passlib.hash import sha256_crypt

def złam_sha256_crypt(hash_docelowy, słownik):
    części = hash_docelowy.split('$')
    
    # Wyodrębnij salt (obsłuż parametr rounds)
    if 'rounds=' in części[2]:
        salt = części[3]
    else:
        salt = części[2]
    
    for hasło in słownik:
        test_hash = sha256_crypt.using(salt=salt).hash(hasło)
        
        if test_hash == hash_docelowy:
            return hasło  # Znalezione: "italy"
    
    return None
```

#### Krok 4: Złam hash Argon2
Dano: `$argon2id$v=19$m=16384,t=2,p=1$salt$hash`

```python
from argon2 import PasswordHasher
from argon2.exceptions import VerifyMismatchError

def złam_argon2(hash_docelowy, słownik):
    ph = PasswordHasher()
    
    for hasło in słownik:
        try:
            ph.verify(hash_docelowy, hasło)
            return hasło  # Znalezione: "1951"
        except VerifyMismatchError:
            continue
    
    return None
```

#### Krok 5: Złam hash z pieprzem
Dano: Hash z nieznanym 1-znakowym pieprzem [a-z] doklejonym

```python
def złam_z_pieprzem(hash_docelowy, słownik, funkcja_hash):
    for hasło in słownik:
        for pieprz in 'abcdefghijklmnopqrstuvwxyz':
            kandydat = hasło + pieprz
            
            if testuj_dopasowanie_hash(kandydat, hash_docelowy, funkcja_hash):
                return kandydat  # Znalezione: "maryannd" + "d"
    
    return None
```

### Oczekiwane rezultaty

| Typ hash'a | Hasło | Czas | Uwagi |
|------------|-------|------|-------|
| Kolizja MD5 | pass862, pass4708 | ~0.01s | Paradoks urodzin |
| MD5-crypt | alibaba | ~2s | Szybki, słaby schemat |
| SHA256-crypt | italy | ~10s | Średnia trudność |
| Argon2 | 1951 | ~600s | Memory-hard, wolny |
| Pieprz (SHA256) | maryannd | ~300s | Mnożnik 26× |

### Wskazówki wydajnościowe
1. **Wstępna kompilacja wzorców**: Optymalizuj ładowanie słownika
2. **Przetwarzanie równoległe**: Użyj multiprocessing dla zadań ograniczonych CPU
3. **Akceleracja GPU**: Użyj hashcat/john do poważnego łamania
4. **Śledzenie postępu**: Pokaż ETA dla długotrwałych ataków
5. **Wczesne zakończenie**: Zatrzymaj natychmiast gdy hasło znalezione